<a id="top"></a>

<a id = '1.0'></a>
<h1 style = "font-family: garamond; font-size: 40px; font-style: normal;background-color: #2ab7ca; color : #fed766; border-radius: 5px 5px;padding:5px;text-align:center; font-weight: bold" >Quick Navigation</h1>

    
* [Dependencies and Configuration](#1)
* [Stage 3: Feature Engineering/Feature Selection](#2)
    * [Multicollinearity and Feature Selection](#31)
        * [Target Distribution](#31)
        * [Using Statsmodels Variance Inflation Factor](#31)
        * [Oh Dear, we have a Multicollinearity Problem](#31)
    * [Save the Data](#31)

# Dependencies and Configuration

In [32]:
import random
from collections import defaultdict
from typing import Dict, List, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import (base, decomposition, linear_model, manifold, metrics,
                     preprocessing)

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [33]:
class global_config:
    
    # File Path
    raw_data = "../data/raw/data.csv"
    processed_data_stage_1 = "../data/processed/data_stage_1.csv"
    processed_data_stage_2 = "../data/processed/data_stage_2.csv"
    processed_data_stage_3 = "../data/processed/data_stage_3.csv"

    # Data Information
    target = ["diagnosis"]
    unwanted_cols = ["id", "Unnamed: 32"]

    # Plotting
    colors = ["#fe4a49", "#2ab7ca", "#fed766", "#59981A"]
    cmap_reversed = plt.cm.get_cmap('mako_r')
    
    # Seed Number
    seed = 1992

    # Cross Validation
    num_folds = 5
    cv_schema = "StratifiedKFold"
    split_size = {"train_size": 0.9, "test_size": 0.1}


def set_seeds(seed: int = 1234) -> None:
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)

In [34]:
config = global_config

# set seeding for reproducibility
_ = set_seeds(seed = config.seed)

# read data
df = pd.read_csv(config.processed_data_stage_2)

# Stage 3: Feature Engineering/Feature Selection

<div class="alert alert-block alert-danger">
<b>Disclaimer!</b> We are fully aware that oftentimes practitioner may accidentally cause data leakage during preprocessing, for example, a subtle yet often mistake is to standardize the whole dataset prior to splitting, or performing feature selection prior to modelling using the information of our response/target variable. However, we can still screen predictors for multicollinearity during EDA phase and have a good intuition on which predictors are highly correlated - subsequently, we will incorporate a feature selection technique in our modelling pipeline. 
</div>

## Multicollinearity and Feature Selection

<div class="alert alert-info" role="alert">
    <b>Motivation:</b> We need feature selection in certain problems for the following reasons:
    <li> Well, one would definitely have heard of the dreaded curse of dimensionality in the journey of learning Machine Learning where having too many predictor/features can lead to overfitting; on the other hand, too many dimensions can cause distance between observations to appear equidistance from one another. observations become harder to cluster — believe it or not, too many dimensions causes every observation in your dataset to appear equidistant from all the others, thereby clogging the model's ability to cluster data points (imagine the horror if you use KNN on 1000 dimensions, all the points will be almost the same distance from each other, poor KNN).
    <li> In case you have access to Google's GPU clusters, you likely want to train your model faster. Reducing the number predictors can aid this process.
    <li> Reducing uninformative features may aid in model's performance, the idea is to remove unnecessary noise from the dataset.
</div>

---

<div class="alert alert-block alert-danger">
<b>Multi-Collinearity:</b> Looking back at our dataset, it is clear to me that there are quite a number of features that are correlated with each other, causing multi-collinearity. Multi-Collinearity is an issue in the history of Linear Models, quoting the statement from <a href="https://stats.stackexchange.com/questions/1149/is-there-an-intuitive-explanation-why-multicollinearity-is-a-problem-in-linear-r"> Is there an intuitive explanation why multicollinearity is a problem in linear regression?</a>
    
> Consider the simplest case where Y is regressed against X and Z and where X and Z are highly positively correlated. Then the effect of X on Y is hard to distinguish from the effect of Z on Y because any increase in X tends to be associated with an increase in Z.

We also note that multi-collinearity is not that big of a problem for non-parametric models such as Decision Tree or Random Forests, however, I will attempt to show that it is still best to avoid in this problem setting.
</div>

<div class="alert alert-warning" role="alert">
    <b> Alert! Alert! Alert! </b> There are many methods to perform feature selection. Scikit-Learn offers some of the following:
    <li> Univariate feature selection.
    <li> Recursive feature elimination.
    <li> Backward Elimination of features using Hypothesis Testing.  
</div>

---

<div class="alert alert-warning" role="alert">
    <b> EMERGENCY! </b> We need to be careful when selecting features before cross-validation. It is therefore, recommended to include feature selection in cross-validation to avoid any "bias" introduced before model selection phase! I decided to use the good old Variance Inflation Factor (VIF) as a way to reduce multicollinearity. Unfortunately, there is no out-of-the-box function to integrate into the <code>Pipeline</code> of scikit-learn. Thus, I heavily modified an existing code in order achieve what I want below.
</div>

A classical way to check for multicollinearity amongst predictors is to calculate the Variable Inflation Factor (VIF). It is simply done by regressing each predictor $\mathrm{x}_i$ against all other predictors $\mathrm{x}_j, j \neq i$. In other words, the VIF for a predictor variable $i$ is given by:

$$\text{VIF}_i = \dfrac{1}{1 - R^{2}_{i}}$$

where $R^{2}_{i}$ is, by definition, the proportion of the variation in the "dependent variable" $\mathrm{x}_i$ that is predictable from the indepedent predictors $\mathrm{x}_j, j \neq i$. Consequently, the higher the $R^2_i$ of a predictor, the higher the VIF, and this indicates there is linear dependence among predictors.

### Using Statsmodels Variance Inflation Factor

Note that we need to perform scaling first before fitting our `ReduceVIF` to get the exact same result as the previous version. In this version, I manually added a hard threshold for the number of features remaining to be 15. This hard coded number can be turned into a parameter (hyperparameter) in our pipeline.

In [35]:
class ReduceVIF(base.BaseEstimator, base.TransformerMixin):
    """The base of the class structure is not implemented by me, however, I heavily modified the class such that it can
    take in numpy arrays and correctly implemented the fit and transform method.
    """

    def __init__(self, thresh=10):
        self.thresh = thresh
        self.feature_names_ = None
        self.predictor_cols = [
            "radius_mean",
            "texture_mean",
            "perimeter_mean",
            "area_mean",
            "smoothness_mean",
            "compactness_mean",
            "concavity_mean",
            "concave points_mean",
            "symmetry_mean",
            "fractal_dimension_mean",
            "radius_se",
            "texture_se",
            "perimeter_se",
            "area_se",
            "smoothness_se",
            "compactness_se",
            "concavity_se",
            "concave points_se",
            "symmetry_se",
            "fractal_dimension_se",
            "radius_worst",
            "texture_worst",
            "perimeter_worst",
            "area_worst",
            "smoothness_worst",
            "compactness_worst",
            "concavity_worst",
            "concave points_worst",
            "symmetry_worst",
            "fractal_dimension_worst",
        ]

    def reset(self):

        self.predictor_cols = [
            "radius_mean",
            "texture_mean",
            "perimeter_mean",
            "area_mean",
            "smoothness_mean",
            "compactness_mean",
            "concavity_mean",
            "concave points_mean",
            "symmetry_mean",
            "fractal_dimension_mean",
            "radius_se",
            "texture_se",
            "perimeter_se",
            "area_se",
            "smoothness_se",
            "compactness_se",
            "concavity_se",
            "concave points_se",
            "symmetry_se",
            "fractal_dimension_se",
            "radius_worst",
            "texture_worst",
            "perimeter_worst",
            "area_worst",
            "smoothness_worst",
            "compactness_worst",
            "concavity_worst",
            "concave points_worst",
            "symmetry_worst",
            "fractal_dimension_worst",
        ]

    def fit(self, X, y=None):
        print("ReduceVIF fit")
        tmp, self.predictor_cols = ReduceVIF.calculate_vif(X, self.predictor_cols, self.thresh)
        self.feature_names_ = self.predictor_cols  # save as an attribute to call later
        # If you notice this code is wrong here, we will return a sequential index no matter what.
        col_index = [self.predictor_cols.index(col_name) for col_name in self.predictor_cols]
        self.col_index = col_index
        self.reset()
        return self

    def transform(self, X, y=None):
        print("ReduceVIF transform")
        # columns = X.columns.tolist()
        # print(X.shape)
        print(self.col_index)
        return X[:, self.col_index]

    @staticmethod
    def calculate_vif(X: Union[np.ndarray, pd.DataFrame], columns: List[str], thresh: float = 10.0):
        """Implements a VIF function that recursively eliminates features.

        Args:
            X (Union[np.ndarray, pd.DataFrame]): [description]
            columns (List[str]): [description]
            thresh (float, optional): [description]. Defaults to 10.0.

        Returns:
            [type]: [description]
        """

        dropped = True
        count = 0
        while dropped and count <= 15:
            column_index = X.shape[1]
            predictor_cols = np.arange(X.shape[1])
            dropped = False
            vif = []
            for var in range(column_index):
                # print(predictor_cols.shape)
                vif.append(variance_inflation_factor(X[:, predictor_cols], var))

            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f"Dropping {maxloc} with vif={max_vif}")
                # X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                X = np.delete(X, maxloc, axis=1)
                columns.pop(maxloc)
                dropped = True
                count += 1
        return X, columns

In [36]:
predictor_cols = df.columns[1:]
transformer = ReduceVIF()
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(df[predictor_cols])
# Only use 10 columns for speed in this example
X = transformer.fit_transform(X)

vif_df = pd.DataFrame({'Predictors': transformer.feature_names_})

ReduceVIF fit
Dropping 0 with vif=3806.1152963979675
Dropping 19 with vif=616.3508614719424
Dropping 1 with vif=325.64131198187516
Dropping 19 with vif=123.25781086343038
Dropping 4 with vif=64.65479584770004
Dropping 7 with vif=35.61751844352034
Dropping 19 with vif=33.96063880508537
Dropping 20 with vif=30.596655364833975
Dropping 1 with vif=25.387829695531387
Dropping 2 with vif=18.843208489973282
Dropping 14 with vif=17.232376192128665
Dropping 7 with vif=16.333806476471736
Dropping 15 with vif=15.510661467365699
ReduceVIF transform
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [37]:
print(f"Remaining Features: {transformer.feature_names_}")
display(vif_df)

Remaining Features: ['texture_mean', 'smoothness_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'texture_se', 'perimeter_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'area_worst', 'smoothness_worst', 'symmetry_worst', 'fractal_dimension_worst']


,Predictors
0,texture_mean
1,smoothness_mean
2,concave points_mean
3,symmetry_mean
4,fractal_dimension_mean
5,texture_se
6,perimeter_se
7,smoothness_se
8,compactness_se
9,concavity_se


### Improved Version of ReduceVIF class

This is an improved version of the previous `ReduceVIF` class. Notice in order to write it "our way", we took the `variance_inflation_factor` from the `statsmodels` library and redefine it here to suit our purpose.

In [38]:
import numpy as np
import pandas as pd
from sklearn import base
from statsmodels.regression.linear_model import OLS

def variance_inflation_factor(exog, idx_kept, vif_idx):
    """Compute VIF for one feature.
    
    Args:
        exog (np.ndarray): Observations
        idx_kept (List[int]): Indices of features to consider
        vif_idx (int): Index of feature for which to compute VIF
    
    Returns:
        float: VIF for the selected feature
    """
    exog = np.asarray(exog)
    
    x_i = exog[:, vif_idx]
    mask = [col for col in idx_kept if col != vif_idx]
    x_noti = exog[:, mask]
    
    r_squared_i = OLS(x_i, x_noti).fit().rsquared
    vif = 1. / (1. - r_squared_i)
    
    return vif

class ReduceVIF(base.BaseEstimator, base.TransformerMixin):
    """The base of the class structure is implemented in https://www.kaggle.com/ffisegydd/sklearn-multicollinearity-class;
    I heavily modified the class such that it can take in numpy arrays and correctly implemented the fit and transform method.
    """

    def __init__(self, thresh=10, max_drop=20):
        self.thresh = thresh
        self.max_drop = max_drop
        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def reset(self):
        """Resets the state of predictor columns after each fold."""

        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def fit(self, X, y=None):
        """Fits the Recursive VIF on the training folds and save the selected feature names in self.feature_names

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """
        
        self.column_indices_kept_, self.feature_names_kept_ = self.calculate_vif(X)
        
        return self

    def transform(self, X, y=None):
        """Transforms the Validation Set according to the selected feature names.

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """

        return X[:, self.column_indices_kept_]

    def calculate_vif(self, X: Union[np.ndarray, pd.DataFrame]):
        """Implements a VIF function that recursively eliminates features.

        Args:
            X (Union[np.ndarray, pd.DataFrame]): [description]

        Returns:
            [type]: [description]
        """
        feature_names = None
        column_indices_kept = list(range(X.shape[1]))
        
        if isinstance(X, pd.DataFrame):
            feature_names = X.columns

        dropped = True
        count = 0
        
        while dropped and count <= self.max_drop:
            dropped = False
            
            max_vif, max_vif_col = None, None
            
            for col in column_indices_kept:
                
                vif = variance_inflation_factor(X, column_indices_kept, col)
                
                if max_vif is None or vif > max_vif:
                    max_vif = vif
                    max_vif_col = col
            
            if max_vif > self.thresh:
                print(f"Droppingggggg {max_vif_col} with vif={max_vif}")
                column_indices_kept.remove(max_vif_col)
                
                if feature_names is not None:
                    feature_names.pop(max_vif_col)
                    
                dropped = True
                count += 1
                
        return column_indices_kept, feature_names


We do a sanity check if this coincides with the previous defined class, and the results are the same.

In [39]:
predictor_cols = df.columns[1:]
transformer = ReduceVIF()
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(df[predictor_cols])
# Only use 10 columns for speed in this example
X = transformer.fit_transform(X)

print(f"Remaining Features: {transformer.column_indices_kept_}")

Droppingggggg 0 with vif=3806.1152963979675
Droppingggggg 20 with vif=616.3508614719424
Droppingggggg 2 with vif=325.64131198187516
Droppingggggg 22 with vif=123.25781086343038
Droppingggggg 6 with vif=64.65479584770004
Droppingggggg 10 with vif=35.61751844352034
Droppingggggg 25 with vif=33.96063880508537
Droppingggggg 27 with vif=30.596655364833975
Droppingggggg 3 with vif=25.387829695531387
Droppingggggg 5 with vif=18.843208489973282
Droppingggggg 21 with vif=17.232376192128665
Droppingggggg 13 with vif=16.333806476471736
Droppingggggg 26 with vif=15.510661467365699
Remaining Features: [1, 4, 7, 8, 9, 11, 12, 14, 15, 16, 17, 18, 19, 23, 24, 28, 29]


We have the remaining indices, and therefore simply use numpy to subset the column indices to get back the original column names that are kept.

In [40]:
vif_df = pd.DataFrame({'Predictors': predictor_cols[transformer.column_indices_kept_]})

In [41]:
display(vif_df)

,Predictors
0,texture_mean
1,smoothness_mean
2,concave points_mean
3,symmetry_mean
4,fractal_dimension_mean
5,texture_se
6,perimeter_se
7,smoothness_se
8,compactness_se
9,concavity_se


### Oh Dear, we have a Multicollinearity Problem

<div class="alert alert-block alert-info">
<b>Using VIF in Modelling Pipeline:</b> At this step, we are just showing how we can remove multicollinear features using VIF; but we will not remove them at this point in time. We will incorporate this feature selection technique in our Cross-Validation pipeline in order to avoid data leakage.
</div>

# Save the Data

In [12]:
df.to_csv(config.processed_data_stage_3, index=False)